# RAG avec Mistral (Retrieval-Augmented Generation)
Fait par les Etudiants du Matser 2 :
- Mohamed DIALLO
- Yaya SANAOGO

**FST-USTTB**

Ce système est conçu pour extraire des informations pertinentes d'un document PDF et générer des réponses à des questions basées sur ces informations.

## Import des packages

In [35]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
import os
import pickle

## Copie du model Mistral 
1. **Chargement de la copie mistral en local (Ollama) :**

Pour des certaines raisons nous avons opter pour faire une copie du model mistral en local (ModelFile) a quelle comme temperature du 1 pour cette copie

*sources* : 
- [Running models with Ollama step-by-step](https://medium.com/@gabrielrodewald/running-models-with-ollama-step-by-step-60b6f6125807) 

- [ollama_custom_model](https://unmesh.dev/post/ollama_custom_model/)

- [How to Customize LLMs with Ollama](https://medium.com/@sumudithalanz/unlocking-the-power-of-large-language-models-a-guide-to-customization-with-ollama-6c0da1e756d9)

In [2]:
# Initialize model mistral
model = Ollama(model="mistral_copy")

## Chargement et Prétraitement des Documents

2. **Chargement du Document PDF avec PyMuPDFLoader:**

Il s'agit de l'option d'analyse PDF la plus rapide. Elle contient des métadonnées détaillées sur le PDF et ses pages, et renvoie un document par page.

source : [Using PyMuPDF](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#using-pymupdf)

Le document PDF, spécifiquement mon CV, est chargé à l'aide de PyMuPDFLoader depuis le chemin : /home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf.

In [14]:
pdf_path = "/home/mohamed/Downloads/2022-13_compressed.pdf"
loader = PyMuPDFLoader(pdf_path)
doc = loader.load()

3. **Découpage du Texte :**

Pour faciliter l'indexation et la recherche, le texte du PDF est découpé en morceaux plus petits. Ce découpage est réalisé avec RecursiveCharacterTextSplitter, utilisant des séparateurs variés (sauts de ligne, espaces, ponctuations). Chaque morceau de texte a une taille maximale de 300 caractères avec un chevauchement de 50 caractères pour conserver le contexte entre les morceaux.

LangChain propose différents types de séparateurs de texte suivant le tableau recapitulatif ci-dessous (premier lien dans le block des sources)
| Nom                               | Classes                                      | Sépare sur                     | Ajoute des métadonnées | Description                                                                                                                                                                                                                       |
|-----------------------------------|----------------------------------------------|-------------------------------|------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Récursif                          | RecursiveCharacterTextSplitter, RecursiveJsonSplitter | Une liste de caractères définis par l'utilisateur |                        | Sépare le texte de manière récursive. Cette méthode essaye de garder les morceaux de texte liés les uns aux autres. C'est la méthode recommandée pour commencer à séparer du texte.                                                |
| HTML                              | HTMLHeaderTextSplitter, HTMLSectionSplitter  | Caractères spécifiques à HTML | ✅                     | Sépare le texte en fonction des caractères spécifiques à HTML. Notamment, cela ajoute des informations pertinentes sur l'origine de ce morceau (basé sur le HTML).                                                                |
| Markdown                          | MarkdownHeaderTextSplitter                   | Caractères spécifiques à Markdown | ✅                     | Sépare le texte en fonction des caractères spécifiques à Markdown. Notamment, cela ajoute des informations pertinentes sur l'origine de ce morceau (basé sur le Markdown).                                                        |
| Code                              | plusieurs langages                           | Caractères spécifiques au code (Python, JS) |                        | Sépare le texte en fonction des caractères spécifiques aux langages de programmation. 15 langues différentes sont disponibles au choix.                                                                                          |
| Token                             | plusieurs classes                            | Tokens                        |                        | Sépare le texte en tokens. Il existe plusieurs manières de mesurer les tokens.                                                                                                                                                    |
| Caractère                         | CharacterTextSplitter                        | Un caractère défini par l'utilisateur |                        | Sépare le texte en fonction d'un caractère défini par l'utilisateur. C'est l'une des méthodes les plus simples.                                                                                                                   |
| [Expérimental] Segmenteur Sémantique | SemanticChunker                             | Phrases                       |                        | Sépare d'abord en phrases. Puis combine celles qui sont adjacentes si elles sont suffisamment similaires sémantiquement. Tiré de Greg Kamradt.                                                                                     |
| AI21 Segmenteur de Texte Sémantique | AI21SemanticTextSplitter                    |                             | ✅                       | Identifie des sujets distincts qui forment des morceaux de texte cohérents et les sépare en conséquence.                                                                                                                           |


sources :

- [Types of Text Splitters](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/#types-of-text-splitters) 

- [Splitting text from languages without word boundaries](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries)

In [15]:
# Définir les séparateurs pour le découpage du texte
separators = [
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
]

In [16]:
# Initialiser le découpeur de texte avec les paramètres spécifiés
text_splitter = RecursiveCharacterTextSplitter(
    separators=separators,
    chunk_size=300, # Taille de chaque chunk en caractères
    chunk_overlap=100, # Chevauchement entre les chunks consécutifs
    length_function=len, # Fonction pour calculer la longueur du texte
    add_start_index=True, # Ajouter l'index de début à chaque chunk
)

In [17]:

chunks = text_splitter.split_documents(doc)
print(f"Split {len(doc)} documents into {len(chunks)} chunks.")

Split 121 documents into 583 chunks.


In [18]:
# Afficher un exemple de contenu de page et de métadonnées pour un chunk
page = chunks[0]
print(page.page_content)
print(page.metadata)

Enquête d’opinion «Que pensent les Malien(ne)s ?»
1
Chèr(e)s ami(e)s de la Friedrich-Ebert-Stiftung 
au Mali,  
 
Depuis 10 ans, nous demandons "Qu'en 
pensent les Malien(ne)s ? Lorsque nous avons 
commencé le Mali-Mètre en 2012, nous 
n'avions pas conscience de la portée qu'il
{'source': '/home/mohamed/Downloads/2022-13_compressed.pdf', 'file_path': '/home/mohamed/Downloads/2022-13_compressed.pdf', 'page': 2, 'total_pages': 121, 'format': 'PDF 1.4', 'title': 'Mise en page 1', 'author': 'IMPRIM COLOR', 'subject': '', 'keywords': '', 'creator': 'QuarkXPress(R) 14.21', 'producer': 'iLovePDF', 'creationDate': 'D:20220512192754Z', 'modDate': 'D:20240726083929Z', 'trapped': '', 'start_index': 0}


## Création et Gestion des Embeddings

4. **Génération des Embeddings :**

Les embeddings, qui sont des représentations vectorielles des morceaux de texte, sont générés à l'aide du modèle sentence-transformers/all-MiniLM-L6-v2 fourni par HuggingFaceEmbeddings.

Pourquoi le modèle sentence-transformers/all-MiniLM-L6-v2 alors qu'on a passé beaucoup de temps à télécharger Mistral et là, on vient nous parler d'un autre modèle voilà la question que l'on c'était poser au tout debut mais en des relectures et quelque recherche on as su que le sentence-transformers c'était juste pour nous aider dans la construction de notre base vectorielle il y en plein d'autre le plus connu reste le all-MiniLM-l6-V2 enfin disons par défaut. 

*sources :*

- [Embedding models](https://ollama.com/blog/embedding-models)

- [Using Langchain, Chroma, and GPT for document-based retrieval-augmented generation](https://developer.dataiku.com/12/tutorials/machine-learning/genai/nlp/gpt-lc-chroma-rag/index.html)

In [19]:
# Récupérer la fonction d'embeddings à partir des ressources du code env
emb_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=emb_model)

2024-07-26 21:23:40.840197: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-26 21:23:41.029399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 21:23:41.119431: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 21:23:41.144748: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 21:23:41.278142: I tensorflow/core/platform/cpu_feature_guar

5. **Cache des Embeddings :**

Pour optimiser les performances, les embeddings sont soit chargés depuis un cache (emb_cache.pkl), soit générés et ensuite enregistrés pour une utilisation future si le cache n'existe pas.

In [20]:
# Chemin vers le répertoire pour enregistrer la base de données Chroma
CHROMA_PATH = "embdb"
EMB_CACHE_PATH = "emb_cache.pkl"

In [21]:
# Charger les embeddings depuis le cache ou les générer si le cache n'existe pas
if os.path.exists(EMB_CACHE_PATH):
    with open(EMB_CACHE_PATH, 'rb') as f:
        cached_embeddings = pickle.load(f)
else:
    # Générer les embeddings pour chaque chunk
    cached_embeddings = embeddings.embed_documents([chunk.page_content for chunk in chunks])
    with open(EMB_CACHE_PATH, 'wb') as f:
        pickle.dump(cached_embeddings, f)

## Indexation et Recherche

6. **Base de Données Chroma :**

Les morceaux de texte, accompagnés de leurs embeddings, sont stockés dans une base de données Chroma. 
Cette base de données est persistée dans le répertoire embdb pour assurer une sauvegarde sur le disque.

In [22]:
# Créer une nouvelle base de données Chroma et ajouter les documents
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
db.add_documents(documents=chunks)

['acb23676-6c19-459b-a446-ceda9db82e83',
 'b66d4326-bf34-455e-842f-e43c232c9bc2',
 '010e5809-27b1-410e-8342-4874ee040b80',
 '5e250f83-c47e-450b-843b-d0ccf1a95db7',
 'a767a3ff-ca83-47bd-a74f-03e735b58e58',
 '3e82048d-56e9-43ff-b210-a54c5492c9f3',
 'dde9debe-a396-4983-9481-bf2e6c250e66',
 '2f6e85ef-c398-4729-8149-1ecc09dd88bf',
 '989e63ff-96de-4f13-9539-32320b764c12',
 '59b7e135-eefe-428f-a788-966124a34f22',
 '0c26be85-d713-4275-9e43-3443fff4ae50',
 '3c0ef3b3-2f2e-4b09-9fd2-23f1674c61c8',
 '7ec57bd6-ac6d-47c4-bdb1-74ab59c24892',
 'efbee8b8-f49c-44ed-8af3-4fb997ac00bc',
 '5ef43a5d-b89b-4dbe-b230-b0e6be536a92',
 'dd74cce0-6edb-41d3-9229-2a5a181c0c31',
 '3f1dcfaf-eae9-4205-8f1d-29d4bfad25ea',
 'a744c17f-3dbd-4bee-a20d-5b51296cbac3',
 'e760f4e5-51bb-4a4e-a10c-750751603dc0',
 'de84b31c-88f4-49a3-a433-5f456471509b',
 '1ae3bbed-5a9d-4b5b-9a47-0eac6bc407cd',
 '6ad70b41-c0ce-4ed3-8842-b1981ded025c',
 'd1ea69b0-5522-4e6b-833a-276a5a24a799',
 '12a5ef43-25c0-4ed4-b5c0-82448022147a',
 '6a651ea9-1f0b-

In [23]:
# Persister la base de données sur le disque
db.persist()
print(f"Enregistrement de {len(chunks)} chunks dans {CHROMA_PATH}.")

Enregistrement de 583 chunks dans embdb.


### Première question :

In [24]:
query_text = "De quoi parle t'on dans ce document ?"

7. **Recherche par Similarité et Ranking:**

Lorsqu'une question est posée, le système effectue une recherche par similarité dans la base de données Chroma.
Les documents les plus pertinents sont récupérés en fonction de leur similarité avec la question avec mecanisque de ranking.

In [25]:
# Retrieve context from DB using similarity search
results = db.similarity_search_with_relevance_scores(query_text, k=3)  # Récupérer plus de résultats pour le ranking
results.sort(key=lambda x: x[1], reverse=True)  # Tri par score de pertinence
top_results = results[:3]  # Prendre les 3 meilleurs résultats

context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in top_results])

## Génération de Réponses avec Compression du Prompt

8. **Modèle de Prompt :**

Un modèle de prompt est défini pour structurer la réponse. Le prompt intègre le contexte extrait des documents pertinents pour guider le modèle Mistral dans la génération de réponses claires et utiles.

9. **Compression du Prompt :**

Avant d'intégrer la question dans le modèle de prompt, une phase de compression du prompt est réalisée. 
Cela implique de réduire la longueur du contexte tout en conservant les informations essentielles. 
Cette étape permet de rester dans les limites de longueur du modèle tout en maintenant la pertinence de la réponse.

In [26]:
# Function to compress prompts
def compress_prompt(context, question):
    # Keep only the most relevant parts of the context
    compressed_context = context[:1000]  # Limiter à 1000 caractères par exemple
    return f"Question: {question}\nContext: {compressed_context}"

In [27]:
# Compress the prompt
compressed_prompt = compress_prompt(context_text, query_text)

In [31]:
# Create prompt using compressed context and query text
prompt_template = ChatPromptTemplate.from_template("""
    Tu t'appelles Okka.
    Tu es un assistant en intelligence artificielle conçu pour aider les utilisateurs en récupérant des informations pertinentes et en générant des réponses basées sur ces informations.
    Ton objectif est de fournir des réponses claires et utiles.
    Repond toujours dans la même langue que la question

    {compressed_prompt}
    """)

In [32]:
# Create prompt using compressed context and query text
prompt_template = ChatPromptTemplate.from_template(
    """
        Tu t'appelles Okka.
        Tu es un assistant en intelligence artificielle conçu pour aider les utilisateurs en récupérant des informations pertinentes et en générant des réponses basées sur ces informations. 
        Ton objectif est de fournir des réponses claires et utiles.

        L'utilisateur a posé la question suivante : "{question}"
        Recherche des documents pertinents dans la base de connaissances et utilise ces informations pour répondre à la question.

        Réponds à la question en utilisant uniquement le contexte suivant :
        {context}
        - -
        Réponds à la question ci-dessous en te basant uniquement sur le contexte fourni, dans la même langue que la question :
        Question : {question}
    """
)

10. **Génération de Réponse :**

La question posée est intégrée dans le modèle de prompt avec le contexte compressé, et la réponse est générée par le modèle Mistral.

In [33]:
# Generate response based on the prompt
prompt = prompt_template.format(context=context_text, question=query_text)

response_text = model.invoke(prompt_template.format(context=context_text, question=query_text))
#response_text = model.invoke(prompt_template.format(compressed_prompt=compressed_prompt))

# Get sources of the relevant documents
sources = [doc.metadata.get("source", None) for doc, _score in top_results]

# Format and return the response
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response:  Le document semble être une étude ou sondage sur la sécurité personnelle, l'opinion des forces de défense et de sécurité, ainsi que la perception du public envers les opérations militaires Européennes, EUCAP et EUTM. Il est donc possible de parler de ces trois sujets dans le document.
Sources: ['/home/mohamed/Downloads/2022-13_compressed.pdf', '/home/mohamed/Downloads/2022-13_compressed.pdf', '/home/mohamed/Downloads/2022-13_compressed.pdf']


### Posons une nouvelle question :

In [34]:
query_text = "Quelles sont les principales réformes engagées et dans quelles domaines ?"
response_text = model.invoke(prompt_template.format(context=context_text, question=query_text))
# Get sources of the relevant documents
sources = [doc.metadata.get("source", None) for doc, _score in top_results]

# Format and return the response
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response:  Bien que le contexte fourni ne mentionne pas explicitement des réformes en cours, il est clair qu'il y a des préoccupations importantes concernant la sécurité, notamment en ce qui concerne les femmes seules qui sortent de nuit. L'analyse du sexe montre que 70% (7 sur 10) des femmes ne sont pas en sécurité en sortant seule la nuit. Cela suggère qu'il y a un besoin de réformes dans le domaine de la sécurité nocturne pour les femmes, pour garantir leur sécurité lorsqu'elles sortent de jour comme de nuit.

En plus de cela, il est indiqué que les forces de défense et de sécurité sont généralement satisfaites, avec 80% des personnes interrogées étant très satisfaits et 18% plutôt satisfaits. Cela suggère qu'il y a une réforme en cours dans le domaine de la gestion des forces de défense et de sécurité.

Enfin, il est mentionné l'EUCAP (European Civilian Assistance and Police Mission) et l'EUTM (European Union Training Mission). Cependant, plus de la moitié (4/5) n'a pas entendu par

# Résultats

Le système RAG développé permet de traiter efficacement un document PDF et d'extraire des informations pertinentes pour répondre à des questions spécifiques. La réponse générée inclut également les sources des documents pour assurer la traçabilité des informations fournies. La phase de compression du prompt permet d'optimiser la pertinence et la concision des réponses.